# 📊 Module 4.3: Portfolio Optimization

**Time:** 5 hours | **Difficulty:** 🔴 Advanced

## Learning Objectives
- ✅ Modern Portfolio Theory
- ✅ Efficient Frontier
- ✅ Sharpe Ratio optimization
- ✅ Risk parity

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

## 1. Generate Sample Returns

In [ ]:
np.random.seed(42)

# Simulated annual returns for 4 assets
assets = ['Stocks', 'Bonds', 'Gold', 'Real Estate']
n_assets = len(assets)

# Expected returns
expected_returns = np.array([0.10, 0.04, 0.06, 0.08])

# Covariance matrix
cov_matrix = np.array([
    [0.0400, 0.0020, 0.0010, 0.0025],
    [0.0020, 0.0036, 0.0005, 0.0010],
    [0.0010, 0.0005, 0.0225, 0.0015],
    [0.0025, 0.0010, 0.0015, 0.0196]
])

print("Expected Returns:", expected_returns)
print("\nCovariance Matrix:")
print(pd.DataFrame(cov_matrix, index=assets, columns=assets))

## 2. Portfolio Metrics

In [ ]:
def portfolio_return(weights, returns):
    return np.dot(weights, returns)

def portfolio_volatility(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def sharpe_ratio(weights, returns, cov_matrix, rf=0.02):
    ret = portfolio_return(weights, returns)
    vol = portfolio_volatility(weights, cov_matrix)
    return (ret - rf) / vol

# Equal weight portfolio
equal_weights = np.array([0.25, 0.25, 0.25, 0.25])

print(f"Equal Weight Portfolio:")
print(f"  Return: {portfolio_return(equal_weights, expected_returns):.2%}")
print(f"  Volatility: {portfolio_volatility(equal_weights, cov_matrix):.2%}")
print(f"  Sharpe: {sharpe_ratio(equal_weights, expected_returns, cov_matrix):.2f}")

## 3. Maximum Sharpe Ratio Portfolio

In [ ]:
def neg_sharpe(weights):
    return -sharpe_ratio(weights, expected_returns, cov_matrix)

constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
bounds = tuple((0, 1) for _ in range(n_assets))
initial = np.array([1/n_assets] * n_assets)

result = minimize(neg_sharpe, initial, method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights = result.x

print("Optimal Portfolio (Max Sharpe):")
for asset, weight in zip(assets, optimal_weights):
    print(f"  {asset}: {weight:.1%}")
print(f"\nExpected Return: {portfolio_return(optimal_weights, expected_returns):.2%}")
print(f"Volatility: {portfolio_volatility(optimal_weights, cov_matrix):.2%}")
print(f"Sharpe Ratio: {sharpe_ratio(optimal_weights, expected_returns, cov_matrix):.2f}")

## 4. Efficient Frontier

In [ ]:
def min_vol_for_return(target_return):
    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': lambda x: portfolio_return(x, expected_returns) - target_return}
    ]
    result = minimize(lambda x: portfolio_volatility(x, cov_matrix), 
                     initial, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.fun if result.success else np.nan

target_returns = np.linspace(0.04, 0.10, 50)
frontier_vols = [min_vol_for_return(r) for r in target_returns]

plt.figure(figsize=(10, 6))
plt.plot(frontier_vols, target_returns, 'b-', linewidth=2, label='Efficient Frontier')
plt.scatter(portfolio_volatility(optimal_weights, cov_matrix), 
           portfolio_return(optimal_weights, expected_returns),
           marker='*', s=200, c='red', label='Max Sharpe')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 📝 Exercise: Add constraints
Add sector constraints (max 50% in any asset)

In [ ]:
# YOUR CODE HERE


---
**Next:** Module 4.4 - Risk Metrics →